# Create Agent

### Initializing Project Client

In [5]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet
from user_functions import user_functions

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str="eastus.api.azureml.ms;fac34303-435d-4486-8c3f-7094d82a0b60;rg-anksing-5269_ai;anksing-evaluate-project", # os.environ["PROJECT_CONNECTION_STRING"],
)

AGENT_NAME = "Seattle Tourist Assistant for bug bash"

# Adding Tools to be used by Agent 
functions = FunctionTool(user_functions)

toolset = ToolSet()
toolset.add(functions)

### Create Agent

In [ ]:
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name=AGENT_NAME,
    instructions="You are a helpful assistant",
    toolset=toolset,
)

print(f"Created agent, ID: {agent.id}")

### Create Thread

In [ ]:
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

### Create Message

In [ ]:
# Create message to thread

MESSAGE = "Hello"

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=MESSAGE,
)
print(f"Created message, ID: {message.id}")

### Execute

In [ ]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Evaluate

### Get data from agent

from azure.ai.evaluation import AIAgentConverter



# Initialize the converter that will be backed by the project.
converter = AIAgentConverter(project_client)

thread_id = thread.id
run_id = run.id

converted_data = converter.convert(thread_id, run_id)

### Run Agent Evaluator

from azure.ai.evaluation import ToolCallAccuracy , AzureOpenAIModelConfiguration

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint="<>",
    api_key="<>",
    api_version="2025-01-01-preview",
    azure_deployment="gpt-4o-mini",
)

